# Model Selection

### Get API Key

In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

- Note: in the classroom, if you print out this `api_key` variable, it is not a real API key (for security reasons).
- If you wish to run this code on your own machine, outside of the classroom, you can still reuse the code that you see in `helper.py`.
- It uses [python-dotenv](https://pypi.org/project/python-dotenv/) library to securely save and load sensitive information such as API keys.

In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

## Mistral Small

Good for simple tasks, fast inference, lower cost.
- classification

In [3]:
prompt = """
Classify the following email to determine if it is spam or not.
Only respond with the exact text "Spam" or "Not Spam". 

# Email:
🎉 Urgent! You've Won a $1,000,000 Cash Prize! 
💰 To claim your prize, please click on the link below: 
https://bit.ly/claim-your-prize
"""

'Spam'

In [4]:
mistral(prompt, model="mistral-small-latest")

## Mistral Medium

Good for intermediate tasks such as language transformation.
- Composing text based on provided context (e.g. writing a customer service email based on purchase information).

In [5]:
prompt = """
Compose a welcome email for new customers who have just made 
their first purchase with your product. 
Start by expressing your gratitude for their business, 
and then convey your excitement for having them as a customer. 
Include relevant details about their recent order. 
Sign the email with "The Fun Shop Team".

Order details:
- Customer name: Anna
- Product: hat 
- Estimate date of delivery: Feb. 25, 2024
- Return policy: 30 days
"""

In [6]:
response_medium = mistral(prompt, model="mistral-medium-latest")

In [7]:
print(response_medium)

Subject: Welcome to The Fun Shop, Anna! Thank you for your first purchase.

Dear Anna,

We are thrilled to welcome you to The Fun Shop community! We are so grateful that you have chosen to make your first purchase with us and we can't wait for you to enjoy your new hat.

Your order is being carefully processed and we wanted to provide you with some details:

* Estimated delivery date: February 25, 2024
* Product: Hat

We hope that you will be completely satisfied with your new hat, but if for any reason you are not, please know that we have a 30-day return policy. If you have any questions or concerns about your order, or if there's anything else we can help you with, please don't hesitate to reach out to our customer service team at [support@thefunshop.com](mailto:support@thefunshop.com).

Thank you again for choosing The Fun Shop. We look forward to serving you in the future and making your shopping experience with us fun and enjoyable!

Best regards,
The Fun Shop Team.

## Mistral Large: 

Good for complex tasks that require advanced reasoning.
- Math and reasoning with numbers.

In [8]:
prompt = """
Calculate the difference in payment dates between the two \
customers whose payment amounts are closest to each other \
in the following dataset. Do not write code.

# dataset: 
'{
  "transaction_id":{"0":"T1001","1":"T1002","2":"T1003","3":"T1004","4":"T1005"},
    "customer_id":{"0":"C001","1":"C002","2":"C003","3":"C002","4":"C001"},
    "payment_amount":{"0":125.5,"1":89.99,"2":120.0,"3":54.3,"4":210.2},
"payment_date":{"0":"2021-10-05","1":"2021-10-06","2":"2021-10-07","3":"2021-10-05","4":"2021-10-08"},
    "payment_status":{"0":"Paid","1":"Unpaid","2":"Paid","3":"Paid","4":"Pending"}
}'
"""

In [9]:
response_small = mistral(prompt, model="mistral-small-latest")

In [10]:
print(response_small)

To solve this, let's first find the payment amounts that are closest to each other. From the dataset, we can see that the payment amounts for C001 are 125.5 and 210.2, for C002 are 89.99 and 54.3, and for C003 is 120.0.

The differences in payment amounts for each customer are:
- C001: |210.2 - 125.5| = 84.7
- C002: |89.99 - 54.3| = 35.69
- C003: Since there's only one payment amount for C003, no difference is calculated.

The smallest difference is between C002's payments. Now, let's find the difference in their payment dates.

The payment dates for C002 are '2021-10-06' and '2021-10-05'. Since dates are in 'YYYY-MM-DD' format, we can subtract them as if they were numbers to find the difference.

Difference in payment dates: '2021-10-06' - '2021-10-05' = 1 day.

So, the difference in payment dates between the two customers whose payment amounts are closest to each other is 1 day.

In [11]:
response_large = mistral(prompt, model="mistral-large-latest")

In [12]:
print(response_large)

To calculate the difference in payment dates between the two customers whose payment amounts are closest to each other, we first need to identify those two customers.

Looking at the payment amounts, the closest two amounts are $125.5 and $120.0, paid by customers C001 and C003 respectively.

The payment date for customer C001 is 2021-10-05 and for customer C003 is 2021-10-07.

The difference between these two dates is 2 days.

## Expense reporting task

In [13]:
transactions = """
McDonald's: 8.40
Safeway: 10.30
Carrefour: 15.00
Toys R Us: 20.50
Panda Express: 10.20
Beanie Baby Outlet: 25.60
World Food Wraps: 22.70
Stuffed Animals Shop: 45.10
Sanrio Store: 85.70
"""

prompt = f"""
Given the purchase details, how much did I spend on each category:
1) restaurants
2) groceries
3) stuffed animals and props
{transactions}
"""

In [14]:
response_small = mistral(prompt, model="mistral-small-latest")
print(response_small)

To calculate the total spend for each category, we can group the transactions as follows:

1) Restaurants:
- McDonald's: $8.40
- Panda Express: $10.20

Total spend on restaurants: $8.40 + $10.20 = $18.60

2) Groceries:
- Safeway: $10.30
- Carrefour: $15.00
- World Food Wraps: $22.70

Total spend on groceries: $10.30 + $15.00 + $22.70 = $48.00

3) Stuffed animals and props:
- Toys R Us: $20.50
- Beanie Baby Outlet: $25.60
- Stuffed Animals Shop: $45.10
- Sanrio Store: $85.70

Total spend on stuffed animals and props: $20.50 + $25.60 + $45.10 + $85.70 = $176.90

So, you spent $18.60 on restaurants, $48.00 on groceries, and $176.90 on stuffed animals and props.

In [15]:
response_large = mistral(prompt, model="mistral-large-latest")
print(response_large)

Based on the purchase details you provided, here's how much you spent on each category:

1) Restaurants:
   - McDonald's: $8.40
   - Panda Express: $10.20
   - World Food Wraps: $22.70
   Total spent on restaurants: $41.30

2) Groceries:
   - Safeway: $10.30
   - Carrefour: $15.00
   Total spent on groceries: $25.30

3) Stuffed animals and props:
   - Toys R Us: $20.50
   - Beanie Baby Outlet: $25.60
   - Stuffed Animals Shop: $45.10
   - Sanrio Store: $85.70
   Total spent on stuffed animals and props: $176.90

## Writing and checking code

In [16]:
user_message = """
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.

Your code should pass these tests:

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
"""

In [17]:
print(mistral(user_message, model="mistral-large-latest"))

Here's a Python solution using a dictionary to store the numbers we've seen so far and their indices. This allows us to check if a number that would sum to the target exists in the dictionary in constant time.

```python
def twoSum(nums, target):
    num_dict = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_dict:
            return [num_dict[complement], i]
        num_dict[num] = i
    return []

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
```

In this code:

- `num_dict` is a dictionary that stores the numbers we've seen so far and their indices.
- We iterate through `nums` using `enumerate` to access both the index `i` and the number `num` at that index.
- For each number, we calculate its complement (the number that would add to `num` to make `target`).
- If this complement is in `num_dict`, we've found our two numbers, so we return their indices.
- If not, we add `n

### Try out the code that the model provided
- Copy the code that the model provided and try running it!

Here is the code that was output at the time of filming:
```Python
def twoSum(nums, target):
    seen = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i
```
- Also try running the assert statements in the original prompt
```Python
assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
```

In [18]:
def twoSum(nums, target):
    seen = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i

In [19]:
assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]

## Natively Fluent in English, French, Spanish, German, and Italian
- This means that you can use Mistral models for more than translating from one language to another.
- If you are a native Spanish speaker, for instance, you can communicate with Mistral models in Spanish for any of your tasks.

In [20]:
user_message = """
Lequel est le plus lourd une livre de fer ou un kilogramme de plume
"""

In [21]:
print(mistral(user_message, model="mistral-large-latest"))

Une livre de fer pèse environ 0,45 kilogramme, tandis qu'un kilogramme de plumes pèse un kilogramme. Donc un kilogramme de plumes est plus lourd qu'une livre de fer.

Cependant, il est important de noter que cette comparaison est un peu trompeuse car une livre et un kilogramme sont des unités de mesure différentes. Si l'on compare les deux matériaux en utilisant la même unité de mesure, on constate que le fer est beaucoup plus dense que les plumes, ce qui signifie qu'un volume donné de fer pèsera beaucoup plus lourd qu'un volume équivalent de plumes. Par exemple, un cube de fer d'un pouce pèsera beaucoup plus lourd qu'un cube d'un pouce de plumes.

### Try it out for yourself
- Try communicating with the Mistral Large model in Spanish
  - (If you need help, you can first translate a prompt from English to Spanish, and then prompt the model in Spanish).

## List of Mistral models that you can call:

You can also call the two open source mistral models via API calls.
Here is the list of models that you can try:
```
open-mistral-7b
open-mixtral-8x7b
mistral-small-latest
mistral-medium-latest
mistral-large-latest
```

For example:
```Python
mistral(prompt, model="open-mixtral-8x7b")
```